## Installation of required dependencies

In [13]:
!pip install langchain
!pip install langchain-ollama
!pip install python-dotenv
!pip install ipython


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


##Interacting with the LLM

In [ ]:
from langchain_ollama import ChatOllama
from IPython.display import display, Markdown
from dotenv import load_dotenv
import os

#load langsmith connectivity environment variables
dotenv=load_dotenv('.env')

#chat with llama3.2 via ollama
chat_with_llama32=ChatOllama(
    base_url="http://localhost:11434",
    model="llama3.2:latest",
    temperature=0.8,
    max_tokens=250
)
#sync invoke
#response=chat_with_llama32.invoke("Describe the sun")

#stream
#for chunk in chat_with_llama32.stream("Describe the sun"):
#    print(chunk.text(), end="")




The sun is the star at the center of our solar system and the primary source of light, heat, and energy for our planet. It is a massive ball of hot, glowing gas, primarily composed of hydrogen and helium. Here are some key facts about the sun:

**Physical Characteristics:**

* The sun has a diameter of approximately 1,392,684 kilometers (865,374 miles), which is about 109 times larger than the diameter of the Earth.
* Its mass is approximately 330,000 times that of the Earth.
* The surface temperature of the sun is about 5,500 degrees Celsius (10,000°F).
* The core temperature is estimated to be around 15,000,000 degrees Celsius (27,000,000°F).

**Appearance:**

* The sun appears as a bright, yellowish-white disk in the sky due to its intense light and heat.
* It has no solid surface, but a layer of gas that makes up about 70% of its mass.
* The sun's corona, which is its outer atmosphere, extends millions of kilometers into space.

**Composition:**

* The sun is primarily composed of 

## Prompt Template

In [12]:
from langchain_core.prompts import PromptTemplate

prompt_template=PromptTemplate.from_template("What does the sun look like in galaxy {galaxy_id}")
print(prompt_template)
prompt=prompt_template.invoke({"galaxy_id":"Andromeda"})
print(prompt)

response=chat_with_llama32.invoke(prompt)
print(response.text)

input_variables=['galaxy_id'] input_types={} partial_variables={} template='What does the sun look like in galaxy {galaxy_id}'
text='What does the sun look like in galaxy Andromeda'
<bound method BaseMessage.text of AIMessage(content="The Andromeda Galaxy, also known as Messier 31 (M31) or NGC 224, is a spiral galaxy located about 2.5 million light-years away from Earth. Since we can't directly observe the Sun's appearance in another galaxy, I'll provide some context and insights based on observations of the galaxy's stars and structures.\n\nThe Andromeda Galaxy is a massive system with hundreds of billions of stars, including many similar to our Sun. The solar-type stars (G-type main-sequence stars like the Sun) make up about 20% of the galaxy's total star population.\n\nIf we imagine a snapshot of the Sun in the Andromeda Galaxy at any given point, it would likely appear as a faint, distant blue-white dot amidst a vast sea of other stars. The Sun's luminosity and surface temperature 

## Chat Prompt Template

In [26]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt_template=ChatPromptTemplate([
    {"role":"system", "content":"You are a quantum compute expert"},
    {"role":"user", "content":"what is the basic idea behind quantum compute at the {grain} level"}
])
prompt=chat_prompt_template.invoke({"grain":"macro"})
response=chat_with_llama32.invoke(prompt)
response

AIMessage(content='At the macro level, the basic idea behind quantum computing can be explained as follows:\n\n**Classical Computing vs. Quantum Computing**\n\nClassical computers use bits to store and process information, where each bit can have a value of either 0 or 1. The presence or absence of a bit determines its state. Classical computers rely on logical operations that manipulate these bits using gates like AND, OR, and NOT.\n\nQuantum computers, on the other hand, use qubits (quantum bits) to store and process information. Qubits can exist in multiple states simultaneously, which is known as superposition. This means a qubit can represent not just 0 or 1 but any linear combination of 0 and 1, like 0.5 or -0.5.\n\n**Quantum Computing Principles**\n\nThe core principles behind quantum computing are:\n\n1. **Superposition**: Qubits exist in multiple states (0, 1, and all possible combinations) simultaneously.\n2. **Entanglement**: Qubits can become "entangled," meaning their prop

In [34]:
from langchain_core.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate

system_message_prompt_template=SystemMessagePromptTemplate.from_template("You are a quantum compute expert")
human_message_prompt_template=HumanMessagePromptTemplate.from_template("what is the basic idea behind quantum compute at the {grain} level")

chat_prompt_template=ChatPromptTemplate([
    system_message_prompt_template,
    human_message_prompt_template
])
prompt=chat_prompt_template.invoke({"grain":"multiverse"})
for chunk in chat_with_llama32.stream(prompt):
    print(chunk.content, end="")
#response


What a fascinating and complex topic! As a quantum computer expert, I'll try to break down the basic idea behind quantum computing at the multiverse level.

**Quantum Computing Background**

Classical computers use bits (0s and 1s) to process information. Quantum computers, on the other hand, use qubits (quantum bits), which can exist in multiple states simultaneously, thanks to the principles of superposition and entanglement. This allows quantum computers to process vast amounts of information in parallel, making them potentially exponentially faster than classical computers for certain problems.

**Multiverse Hypothesis**

The multiverse hypothesis proposes that our universe is just one of many universes that exist within a larger multidimensional space. Each universe might have its own set of physical laws and constants, potentially leading to an infinite number of parallel universes or branches of reality.

**Quantum Computing at the Multiverse Level**

Now, let's dive into the mu

## Sequential Chaining

In [43]:
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

#runnable
chat_with_llama32=ChatOllama(
    model='llama3.2',
    temperature=0.9,
    max_tokens=250
)

#runnable
system_message_prompt_template=SystemMessagePromptTemplate.from_template("You are a quantum compute expert")
human_message_prompt_template=HumanMessagePromptTemplate.from_template("what is the basic idea behind quantum compute at the {grain} level")

chat_prompt_template=ChatPromptTemplate([
    system_message_prompt_template,
    human_message_prompt_template
])


#build chain 1
chain1 = chat_prompt_template | chat_with_llama32 | StrOutputParser()

#runnable
prompt_template = PromptTemplate.from_template(
    '''
        Analyze the following text {response} and extract headings
    '''
)

#build chain 2
chain2 = {'response' : chain1} | prompt_template | chat_with_llama32 | StrOutputParser()

#invoke the chain
chain3 = chain1 | chain2
result=chain3.invoke({"grain":"molecular"})

print(result)

Here are the main sections or topics discussed in the text:

1. **Quantum Gates**: Discussion on quantum gates, their similarity to classical logic gates, and their use in implementing quantum algorithms such as the Quantum Fourier Transform (QFT) and Grover's algorithm.

2. **Quantum Error Correction**: Explanation of the importance of error correction in quantum computing due to qubits' sensitivity to environmental factors, and discussion of promising approaches like topological quantum error correction (tQEC) and surface codes.

3. **Quantum Algorithm Development**: Overview of notable quantum algorithms such as Shor's algorithm, Grover's algorithm, and the Quantum Approximate Optimization Algorithm (QAOA), and their applications in solving complex problems.

4. **Quantum Computing Hardware**: Discussion on the challenges and promising approaches for developing practical hardware for molecular-scale qubits, including superconducting qubits, trapped ions, and topological quantum comp

## Parallel Execution

In [ ]:
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnableParallel

#runnable
chat_with_llama32=ChatOllama(
    model='llama3.2',
    temperature=0.9,
    max_tokens=250
)

#runnable
system_message_prompt_template=SystemMessagePromptTemplate.from_template("You are a quantum compute expert")
human_message_prompt_template=HumanMessagePromptTemplate.from_template("what is the basic idea behind quantum compute at the {grain} level")

chat_prompt_template=ChatPromptTemplate([
    system_message_prompt_template,
    human_message_prompt_template
])


#runnable
chat_with_llama3=ChatOllama(
    model='llama3.2',
    temperature=0.9,
    max_tokens=250
)

#runnable
system_message_prompt_template=SystemMessagePromptTemplate.from_template("You are a quantum compute expert")
human_message_prompt_template=HumanMessagePromptTemplate.from_template("what is the basic idea behind quantum compute at the {grain} level")

chat_prompt_template=ChatPromptTemplate([
    system_message_prompt_template,
    human_message_prompt_template
])

#build chain 2
chain1 = chat_prompt_template | chat_with_llama3 | StrOutputParser()


runnable_parallel=RunnableParallel({
    "llama32":chain1,
    "llama3":chain2
})
response=runnable_parallel.invoke({"grain":"molecule"})

print(response['llama32'])
print(f'***************')
print(response['llama3'])


***************
**Expanded Analysis**

Molecular-level quantum computing is a promising field that leverages the properties of quantum mechanics to perform calculations and operations on data at the molecular level. By harnessing the unique properties of individual molecules as qubits, researchers can develop new computational tools for simulating chemical reactions, structural analysis, and other molecular processes.

The concept of qubits in quantum computing is crucial to understanding molecular-level quantum computing. Unlike classical bits, which can only exist in a 0 or 1 state, qubits can exist in multiple states simultaneously, known as superposition. This property allows for the manipulation of complex quantum systems, enabling simulations and calculations that are beyond the capabilities of classical computers.

Quantum operations, such as rotations and entanglement, play a vital role in manipulating these quantum states. Rotations involve applying specific sets of quantum ga

## Runnable Lambda

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnableLambda

##Build chain1
#runnable
chat_with_llama32=ChatOllama(
    model='llama3.2',
    temperature=0.9,
    max_tokens=250
)

#runnable
system_message_prompt_template=SystemMessagePromptTemplate.from_template("You are a quantum compute expert")
human_message_prompt_template=HumanMessagePromptTemplate.from_template("what is the basic idea behind quantum compute at the {grain} level")
chat_prompt_template=ChatPromptTemplate([
    ('system', "You are a quantum compute expert"),
    ('user',"what is the basic idea behind quantum compute at the {grain} level")
])

#chain1
chain1 = chat_prompt_template | chat_with_llama32 | StrOutputParser()



##Build chain2
#runnable
chat_with_llama3=ChatOllama(
    model='llama3',
    temperature=0.9,
    max_tokens=250
)
chat_with_qwen=ChatOllama(
    model='qwen3:latest',
    temperature=0.9,
    max_tokens=250
)

#runnable
prompt_template=PromptTemplate.from_template(
    '''
        Analyze the following : {response} and provide a bullet-point summary
    '''
)

def llm_selector(response):
    print(len(str(response)))
    if (len(str(response)) > 300):
        return chat_with_qwen
    return chat_with_llama3

#build chain 2
chain2 = {"response": chain1} | prompt_template | RunnableLambda(llm_selector) | StrOutputParser()

response=chain2.invoke({"grain":"micro"})
print(response)

2727
<think>
Okay, the user provided a detailed explanation of quantum computing, focusing on the microscopic level and key concepts like qubits, superposition, interference, and components like superconducting circuits and ion traps. They also mentioned limitations such as noise, scalability, and energy consumption. The user wants a bullet-point summary of this information.

First, I need to extract the main points from each section. Let me go through each paragraph one by one.

Starting with the introduction: quantum computing uses microscopic particles (atoms, photons) and quantum mechanics principles. The key points here are the use of particles and quantum principles.

Next is Quantum Bits (Qubits): classical bits vs qubits, superposition, ability to process info in parallel. So, the bullet points should mention superposition and parallel processing.

Then Superposition: entanglement, linked particles, creating superposition states. Need to highlight entanglement and how it's used

##Chain Decorator

In [17]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import chain

##Build chain1
#runnable
chat_with_llama32=ChatOllama(
    model='llama3.2',
    temperature=0.9,
    max_tokens=250
)

#runnable
system_message_prompt_template=SystemMessagePromptTemplate.from_template("You are a quantum compute expert")
human_message_prompt_template=HumanMessagePromptTemplate.from_template("what is the basic idea behind quantum compute at the {grain} level")
chat_prompt_template=ChatPromptTemplate([
    ('system', "You are a quantum compute expert"),
    ('user',"what is the basic idea behind quantum compute at the {grain} level")
])

#chain1
chain1 = chat_prompt_template | chat_with_llama32 | StrOutputParser()



##Build chain2
#runnable
chat_with_llama3=ChatOllama(
    model='llama3',
    temperature=0.9,
    max_tokens=250
)
chat_with_qwen=ChatOllama(
    model='qwen3:latest',
    temperature=0.9,
    max_tokens=250
)

#runnable
prompt_template=PromptTemplate.from_template(
    '''
        Analyze the following : {response} and provide a bullet-point summary
    '''
)

@chain
def llm_selector(response):
    print(len(str(response)))
    if (len(str(response)) > 300):
        return chat_with_qwen
    return chat_with_llama3

#build chain 2
chain2 = {"response": chain1} | prompt_template | llm_selector | StrOutputParser()

response=chain2.invoke({"grain":"micro"})
print(response)

3256
<think>
Okay, let me try to figure out how to approach this query. The user provided a detailed explanation of quantum computing concepts and is asking for a bullet-point summary and any clarifications. 

First, I need to make sure I understand the original content. The explanation covers qubits, superposition, entanglement, quantum gates, algorithms like Shor's and Grover's, and the architecture components. The user wants a concise summary in bullet points and maybe expand on something specific.

Wait, the user also asked, "Is there anything specific you'd like me to expand on or clarify?" So maybe they want the summary first and then an opportunity to ask for more details. But the way the query is phrased, they might be expecting the summary as part of their request. Let me check the original message again.

The user says: "Analyze the following... and provide a bullet-point summary." Then they ask if there's anything specific to expand on. So the main task is to create a bullet